In [ ]:
{-# LANGUAGE BangPatterns, ScopedTypeVariables #-}
import Control.Monad
import Control.Monad.Primitive
import qualified Data.Vector.Unboxed as U

import Numeric.SpecFunctions
import Numeric.MathFunctions.Constants
import Numeric.MathFunctions.Comparison
import Numeric.Polynomial

import Text.Printf(printf)

import IHaskell.Display
import Graphics.Rendering.Chart.Backend.Cairo
import Graphics.Rendering.Chart.Easy hiding (within)

import Debug.Trace

:l NB/Plot

In [ ]:
logGammaLL :: Double -> Double
logGammaLL x
    | x <= 0    = m_pos_inf
    | x <  1    = lanczos (1+x) - log x
    
    | x >= 1 && x < 2 =  lanczos (1+x) - log x
    | otherwise = lanczos x
    where
      -- Evaluate Lanczos approximation for γ=6
      lanczos z = fini 
                $ U.foldl' go (L 0 (z+7)) a
        where
          fini (L l _)   = log (l+a0) + log m_sqrt_2_pi - z65 + (z-0.5) * log z65
          go   (L l t) k = L (l + k / t) (t-1)
          z65 = z + 6.5
      a0  = 0.9999999999995183
      a   = U.fromList [ 0.1659470187408462e-06
                       , 0.9934937113930748e-05
                       , -0.1385710331296526
                       , 12.50734324009056
                       , -176.6150291498386
                       , 771.3234287757674
                       , -1259.139216722289
                       , 676.5203681218835
                       ]
data L = L {-# UNPACK #-} !Double {-# UNPACK #-} !Double

logGamma15 :: Double -> Double
logGamma15 z
  = prefix * y + prefix * r
  where
    y = 0.52815341949462890625
    r = evaluateEvenPolynomialL zm1 coefP
      / evaluateEvenPolynomialL zm1 coefQ
    zm1    = z - 1
    zm2    = z - 2
    prefix = zm1 * zm2
    coefP = [  0.490622454069039543534e-1
         , -0.969117530159521214579e-1
         , -0.414983358359495381969e0
         , -0.406567124211938417342e0
         , -0.158413586390692192217e0
         , -0.240149820648571559892e-1
           , -0.100346687696279557415e-2
         ]
    coefQ = [ 0.1e1
            , 0.302349829846463038743e1
            , 0.348739585360723852576e1
            , 0.191415588274426679201e1
            , 0.507137738614363510846e0
            , 0.577039722690451849648e-1
            , 0.195768102601107189171e-2 ]

logGamma20 :: Double -> Double
logGamma20 z 
  = prefix * y + prefix * r
  where
    prefix = zm1 * zm2
    y   = 0.452017307281494140625
    r   = evaluateEvenPolynomialL (-zm2) coefP
        / evaluateEvenPolynomialL (-zm2) coefQ
    zm1 = z - 1
    zm2 = z - 2
    coefP = [ -0.292329721830270012337e-1
            , 0.144216267757192309184e0
            ,-0.142440390738631274135e0
            , 0.542809694055053558157e-1
            , -0.850535976868336437746e-2
            ,0.431171342679297331241e-3]
    coefQ = [ 0.1e1
            , -0.150169356054485044494e1
            , 0.846973248876495016101e0
            , -0.220095151814995745555e0
            , 0.25582797155975869989e-1
            , -0.100666795539143372762e-2
            , -0.827193521891290553639e-6]


logGamma15 1.2
logGammaL  1.2

In [ ]:
toRenderable $
  plotFunctions [logGammaL,logGamma15,logGamma20] (1, 2)
toRenderable $
  plotFunctions [\x -> logGamma15 x / logGammaL x
                , \x -> logGamma20 x / logGammaL x
                ] (1+1e-6, 2-1e-6)

# Gamma function

In [ ]:
xs <- readFile "data/gamma-17+5.dat"
let vals = map ((\[x,y] -> (read x :: Double, read y :: Double)) . words) $ lines xs

toRenderable $
  let pts = [ (logBase 10 x, d)
            | (x,y) <- vals
            , let d = logBase 10 $ relativeError y (logGamma x)
            , not (isNaN d || isInfinite d)
            ]
  in layout_plots .~ [toPlot $ plot_lines_values .~ [pts] $ def
       , toPlot $ plot_lines_values .~ [[(logBase 10 x,logBase 10 (m_epsilon/2)) | (x,_) <- vals]]
           $ plot_lines_style .~ (line_color .~ opaque red $ def) $ def
  ]
    $ layout_title .~ "Gamma function precision: AS249"
    $ def

toRenderable $
  let pts = [ (logBase 10 x, d)
            | (x,y) <- vals
            , let d = logBase 10 $ relativeError y (logGammaL x)
            , not (isNaN d || isInfinite d)
            ]
  in layout_plots .~ [toPlot $ plot_lines_values .~ [pts] $ def
       , toPlot $ plot_lines_values .~ [[(logBase 10 x,logBase 10 (m_epsilon/2)) | (x,_) <- vals]]
           $ plot_lines_style .~ (line_color .~ opaque red $ def) $ def
           ]
      $ layout_title .~ "Gamma function precision: Lanczos"
    $ def

toRenderable $
  let pts = [ (logBase 10 x, d)
            | (x,y) <- vals
            , let d = logBase 10 $ relativeError y (logGammaLL x)
            , not (isNaN d || isInfinite d)
            ]
  in layout_plots .~ 
       [toPlot $ plot_lines_values .~ [pts] $ def
       , toPlot $ plot_lines_values .~ [[(logBase 10 x,logBase 10 (m_epsilon/2)) | (x,_) <- vals]]
           $ plot_lines_style .~ (line_color .~ opaque red $ def) $ def
--       , toPlot $ plot_lines_values .~ [[(logBase 10 1,-16),(logBase 10 1, -12)]]
--           $ plot_lines_style .~ (line_color .~ opaque red $ def) $ def
--       , toPlot $ plot_lines_values .~ [[(logBase 10 2,-16),(logBase 10 2, -12)]]
--           $ plot_lines_style .~ (line_color .~ opaque red $ def) $ def
       ]
    $ layout_title .~ "Gamma function precision: Lanczos"
    $ def

# Lanczos approximation

In [ ]:
cheb 0 α = case α of
  0 -> 1
  _ -> 0
cheb 1 α = case α of
  1 -> 1
  _ -> 0
cheb k α = 2 * cheb (k - 1) (α - 1) - cheb (k - 2) α
  
coef_Fα α γ = sqrt 2 / pi *  exp (logGammaL (α + 0.5)) *  (α+γ+0.5)**(-(α+0.5)) * exp (α+γ+0.5)
ρ γ k = sum [cheb (2*k) (2*α) * coef_Fα (fromIntegral α) γ | α<- [0..k] ]

In [ ]:
mapM_ (print . ρ 1) [0 .. 5]

In [ ]:
mapM_ (print . ρ 1.5) [0 .. 5]

In [ ]:
mapM_ (print . ρ 1.5) [0 .. 5]

In [ ]:
relativeError 23.0258509298827352372657886135174 (logGammaL 0.000000000100000000000000003643219731549774)

In [ ]:
logGammaL 1
logGammaL 2
logGammaL 100